# 🚗 Smart Urban Parking: Dynamic Pricing Simulation

This notebook implements real-time dynamic pricing for 14 urban parking lots using historical and simulated data. Models are built using Pandas, Numpy, and Pathway, and visualized in real-time with Bokeh.

**Model 1:**
- Linear Price Model

**Model 2:**
- Demand-Based Pricing

**Model 3:**
- Competitive Pricing

All prices are updated based on occupancy, queue length, traffic, and more.


# Model 1 :

## 🧩 Step 1: Load & Prepare Dataset

In [ ]:
# Importing necessary libraries
import pandas as pd
import numpy as np

# Load the original dataset
df = pd.read_csv("/content/dataset.csv")

# Combine date and time columns into one timestamp
df['Timestamp'] = pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'], format='%d-%m-%Y %H:%M:%S')

# Sort data by time to ensure streaming consistency
df = df.sort_values('Timestamp').reset_index(drop=True)

# Save a simplified version for Pathway streaming (can be enriched later)
df[['Timestamp', 'Occupancy', 'Capacity']].to_csv("parking_stream.csv", index=False)
df.head()


,ID,SystemCodeNumber,Capacity,Latitude,Longitude,Occupancy,VehicleType,TrafficConditionNearby,QueueLength,IsSpecialDay,LastUpdatedDate,LastUpdatedTime,Timestamp
0,0,BHMBCCMKT01,577,26.144536,91.736172,61,car,low,1,0,04-10-2016,07:59:00,2016-10-04 07:59:00
1,5248,BHMNCPHST01,1200,26.140014,91.731000,237,bike,low,2,0,04-10-2016,07:59:00,2016-10-04 07:59:00
2,3936,BHMMBMMBX01,687,20.000035,78.000003,264,car,low,2,0,04-10-2016,07:59:00,2016-10-04 07:59:00
3,6560,BHMNCPNST01,485,26.140048,91.730972,249,car,low,2,0,04-10-2016,07:59:00,2016-10-04 07:59:00
4,17056,Shopping,1920,26.150504,91.733531,614,cycle,low,2,0,04-10-2016,07:59:00,2016-10-04 07:59:00


## 🔁 Step 2: Setup Real-Time Stream with Pathway

In [ ]:
# Installing Pathway Bokeh as guided
!pip install pathway bokeh --quiet
import pathway as pw

# Define the schema expected by Pathway
class ParkingSchema(pw.Schema):
    Timestamp: str
    Occupancy: int
    Capacity: int

# Simulate real-time stream from CSV
data = pw.demo.replay_csv("parking_stream.csv", schema=ParkingSchema, input_rate=1000)


## 🕒 Step 3: Add Timestamp and Windowing Logic

In [ ]:

# Define timestamp format
fmt = "%Y-%m-%d %H:%M:%S"

# Add parsed datetime and logical day group
data_with_time = data.with_columns(
    t = data.Timestamp.dt.strptime(fmt),
    day = data.Timestamp.dt.strptime(fmt).dt.strftime("%Y-%m-%dT00:00:00")
)


## 📈 Step 4: Model 1 - Baseline Linear Pricing

In [ ]:

# Create a daily window and calculate occupancy-based linear price
import datetime

delta_window = (
    data_with_time.windowby(
        pw.this.t,
        instance=pw.this.day,
        window=pw.temporal.tumbling(datetime.timedelta(days=1)),
        behavior=pw.temporal.exactly_once_behavior()
    )
    .reduce(
        t=pw.this._pw_window_end,
        occ_max=pw.reducers.max(pw.this.Occupancy),
        cap=pw.reducers.max(pw.this.Capacity)
    )
    .with_columns(
        price=10 + 2.0 * (pw.this.occ_max / pw.this.cap)  # Linear pricing formula
    )
)


## 📊 Step 5: Visualize Prices with Bokeh

In [ ]:
# Importing Libraries for Visualisation
import bokeh.plotting
import panel as pn
pn.extension()

# Price_plotter function to plot graph between "Price of parking " and "Dates"
def price_plotter(source):
    fig = bokeh.plotting.figure(
        height=400,
        width=800,
        title="Pathway: Daily Parking Price",
        x_axis_type="datetime"
    )
    fig.line("t", "price", source=source, line_width=2, color="navy")
    fig.circle("t", "price", source=source, size=6, color="red")
    return fig

viz = delta_window.plot(price_plotter, sorting_col="t")
pn.Column(viz).servable()


Column
    [0] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)

In [ ]:
# Run Pathway pipeline
pw.run()

# Model 2 :

## 🧠 Step 7: Model 2 - Demand-Based Pricing

In [ ]:
# Extended schema
class ExtendedParkingSchema(pw.Schema):
    Timestamp: str
    Occupancy: int
    Capacity: int
    QueueLength: int
    Traffic: int
    SpecialDay: int
    VehicleType: str

# Replay stream for Model 2
data2 = pw.demo.replay_csv("/content/dataset.csv", schema=ExtendedParkingSchema, input_rate=1000)

# Add time and group per day
data2_with_time = data2.with_columns(
    t = data2.Timestamp.dt.strptime(fmt),
    day = data2.Timestamp.dt.strptime(fmt).dt.strftime("%Y-%m-%dT00:00:00")
)

## ▶  Aggregating basic stats, Compute averages, Compute demand score hence final price

In [ ]:
# Step 1: Aggregate basic stats using reduce
aggregated = (
    data2_with_time.windowby(
        pw.this.t,
        instance=pw.this.day,
        window=pw.temporal.tumbling(datetime.timedelta(days=1)),
        behavior=pw.temporal.exactly_once_behavior()
    )
    .reduce(
        t=pw.this._pw_window_end,
        occ_sum=pw.reducers.sum(pw.this.Occupancy),
        queue_sum=pw.reducers.sum(pw.this.QueueLength),
        traffic_sum=pw.reducers.sum(pw.this.Traffic),
        count=pw.reducers.count(),
        capacity=pw.reducers.max(pw.this.Capacity),
        special=pw.reducers.max(pw.this.SpecialDay),
    )
)

# Step 2: Compute averages
with_avgs = aggregated.with_columns(
    occ_avg = pw.this.occ_sum / pw.this.count,
    queue_avg = pw.this.queue_sum / pw.this.count,
    traffic_avg = pw.this.traffic_sum / pw.this.count
)

# Step 3: Compute demand score
with_demand = with_avgs.with_columns(
    demand = (
        0.5 * (pw.this.occ_avg / pw.this.capacity) +
        0.2 * pw.this.queue_avg -
        0.1 * pw.this.traffic_avg +
        0.3 * pw.this.special +
        0.4 * 1.0  # Static weight for vehicle type
    )
)

# Step 4: Compute final price
demand_window = with_demand.with_columns(
    price = 10 * (1 + pw.this.demand / 5)
)


## ♦ Visualize demand-based pricing model

In [ ]:

# Visualize demand-based pricing model
viz2 = demand_window.plot(price_plotter, sorting_col="t")
pn.Column(viz2).servable()


Column
    [0] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)

## ▶️ Run the Pathway Engine

In [ ]:
# Run Pathway pipeline
pw.run()